# Создание и запуск RAG

### Обработка документов

**Проект посвящен созданию RAG (Retrieval-Augmented Generation) с нуля. Мы будем использовать библиотеку Transformers для загрузки языковой модели (LLM), а также рассмотрим реализацию FAISS для эффективного поиска и обработки данных.**

**Цель проекта: помочь пользователю разобраться в интересующих его вопросах, связанных с искусством. Для этого будет использована архитектура RAG (Retrieval-Augmented Generation), которая позволит извлекать релевантную информацию из научных статей и книг. Затем языковая модель (LLM) обработает полученные данные и оформит вывод в удобном и понятном для пользователя виде.**

**Первым шагом загрузим все наши PDF-файлы, из которых будем извлекать необходимую информацию. Для удобства работы перенесем все файлы в список (list) в Python.**

In [1]:
import os
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

pdf_list = []

for pdf in os.listdir('Art'): # место хранения данных
    if os.path.isfile(os.path.join('Art', pdf)): # место хранения данных
        pdf_list.append(pdf)

print(pdf_list)

['Gardners Art Through the Ages The Western Perspective, Volume I,.pdf', 'History_of_Art.pdf', 'Transformative Art Movements and the Paintings That Inspired Them - 2013.pdf', 'Vasari Giorgio_The_Lives_of_the_Artists_Oxford.pdf']


**Для преобразования страниц PDF в текст воспользуемся библиотекой Fitz (также известной как PyMuPDF). Дополнительно добавим метаданные, такие как номер страницы и название документа, чтобы упростить дальнейший анализ и обработку данных.**

In [2]:
import fitz
import re

# Избавляемся от ненужных форматов
def formatting_text(text: str) -> str:
    text = text.replace('\n', ' ').strip()
    text = re.sub(r'[\u2002\u2003\xa0]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def open_and_read_pdf(pdf_list: list) -> list[dict]:
    pdf_file = 0
    pages_and_texts = []
    
    for pdf_path in pdf_list:
        doc = fitz.open('Art/' + pdf_path) # место хранения данных
        pdf_file += 1
        
        for page_number, page in tqdm(enumerate(doc)):
            text = page.get_text()
            text = formatting_text(text)
            replacements = {
                r'\x11': '',
                r'[A-Z](\| |\|)': ' ',
                r'\ue004': ' '}
            for pattern, replacement in replacements.items():
                text = re.sub(pattern, replacement, text)
            pages_and_texts.append({
                'pdf_file': pdf_path, # название файла
                'page_number': page_number, # номер страницы
                'page_char_count': len(text), # количество символов 
                'page_word_count': len(text.split(' ')), # количество слов
                'page_sentence_count_raw': len(text.split('. ')), # количество предложений
                'page_token_count': len(text) / 4, # количество токенов
                'text': text}) # текст
            
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_list=pdf_list)
pages_and_texts[6:8]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[{'pdf_file': 'Gardners Art Through the Ages The Western Perspective, Volume I,.pdf',
  'page_number': 6,
  'page_char_count': 1905,
  'page_word_count': 328,
  'page_sentence_count_raw': 6,
  'page_token_count': 476.25,
  'text': 'v CONTENTS Preface xi INTRODUCTION What Is Art History? 1 Art History in the 21st Century 2 Different Ways of Seeing 13 1 Art in the Stone Age 15 FRAMING THE ERA The Dawn of Art 15 TIMELINE 16 Paleolithic Art 16 Neolithic Art 23 ■ PROBLEMS AND SOLUTIONS: How to Represent an Animal 17 ■ PROBLEMS AND SOLUTIONS: Painting in the Dark 20 ■ A SECOND OPINION: The Meaning of Paleolithic Art 21 ■ ART AND SOCIETY: The Neolithic Temple at Göbekli Tepe 24 MAP 1-1 Stone Age sites in Europe 16 MAP 1-2 Neolithic sites in Anatolia and Mesopotamia 24 THE BIG PICTURE 29 2 Ancient Mesopotamia and Persia 31 FRAMING THE ERA Pictorial Narration in Ancient Sumer 31 TIMELINE 32 Mesopotamia 32 Persia 50 ■ RELIGION AND MYTHOLOGY: The Gods and Goddesses of Mesopotamia 34 ■ PROBLEMS AN

**Преобразуем полученные данные в DataFrame с помощью библиотеки pandas. Это позволит нам удобно структурировать информацию, а также провести анализ и просмотреть статистические данные о нашем наборе.**

In [3]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.sample(5)

,pdf_file,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
1057,Gardners Art Through the Ages The Western Pers...,1057,6176,919,143,1544.00,BIBLIOGRAPHY This list of books is very select...
1553,History_of_Art.pdf,461,3999,672,37,999.75,decorative gold background and an architectura...
1574,History_of_Art.pdf,482,2132,354,19,533.00,"13.2). Of Giotto s surviving murals, those in ..."
2434,Transformative Art Movements and the Paintings...,158,3379,550,26,844.75,157 CONTEXT Piety and passion IN COMPARISON WI...
3188,Vasari Giorgio_The_Lives_of_the_Artists_Oxford...,510,2284,400,4,571.00,484 MICHELANGELO to write and deliver the fune...


In [4]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,3302.00,3302.00,3302.00,3302.00,3302.00
mean,475.77,3329.10,541.48,28.81,832.28
std,321.39,2035.33,320.31,33.24,508.83
min,0.00,0.00,1.00,1.00,0.00
25%,206.00,2157.50,355.00,11.00,539.38
50%,416.00,2966.00,485.00,24.00,741.50
75%,724.75,4360.00,705.75,39.00,1090.00
max,1183.00,16445.00,2640.00,409.00,4111.25


**Круто, теперь у нас есть DataFrame, в котором много информации о тексте. NLP инженеры очень любят большое колличество информации о тексте!**

**Давайте теперь разобъем текст на чанки. Для начала, нам нужно сделать предложения. Для этого воспользуемся sentenizer'ом от spacy.**

In [5]:
from spacy.lang.en import English

nlp = English()
nlp.add_pipe('sentencizer')

for item in tqdm(pages_and_texts):
    item['sentences'] = list(nlp(item['text']).sents)
    item['sentences'] = [str(sentence) for sentence in item['sentences']]
    item['page_sentences_count_spacy'] = len(item['sentences'])

  0%|          | 0/3302 [00:00<?, ?it/s]

In [6]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentences_count_spacy
count,3302.00,3302.00,3302.00,3302.00,3302.00,3302.00
mean,475.77,3329.10,541.48,28.81,832.28,27.78
std,321.39,2035.33,320.31,33.24,508.83,27.95
min,0.00,0.00,1.00,1.00,0.00,0.00
25%,206.00,2157.50,355.00,11.00,539.38,11.00
50%,416.00,2966.00,485.00,24.00,741.50,23.50
75%,724.75,4360.00,705.75,39.00,1090.00,38.00
max,1183.00,16445.00,2640.00,409.00,4111.25,356.00


**Отлично! Продолжаем работу, разбивая текст на чанки. В данном случае мы остановимся на разбиении по 9 предложений в одном чанке. Такой выбор является оптимальным, поскольку в дальнейшем мы будем использовать модель all-mpnet-base-v2 для создания эмбеддингов, а этот размер чанков обеспечивает баланс между сохранением контекста и эффективностью обработки.**

In [7]:
num_sentence_chunk_size = 9

def split_list(slice_size: int, input_list: list) -> list[list[str]]:
    return [input_list[i::i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(pages_and_texts):
    item['sentence_chunks'] = split_list(slice_size=num_sentence_chunk_size,
                                        input_list=item['sentences'])
    item['num_chunks'] = len(item['sentence_chunks'])

  0%|          | 0/3302 [00:00<?, ?it/s]

In [8]:
pages_and_texts[18]

{'pdf_file': 'Gardners Art Through the Ages The Western Perspective, Volume I,.pdf',
 'page_number': 18,
 'page_char_count': 3853,
 'page_word_count': 638,
 'page_sentence_count_raw': 32,
 'page_token_count': 963.25,
 'text': '1 What is art history? Except when referring to the modern academic discipline, people do not often \xadjuxtapose the words art and history. They tend to think of history as the record and interpretation of past human events, particularly social and political events. By contrast, most think of art, quite cor- rectly, as part of the present—as something people can see and touch. Of course, people cannot see or touch history’s vanished human events, but a visible, tangible artwork is a kind of persisting event. One or more artists made it at a certain time and in a specific place, even if no one now knows who, when, where, or why. Although created in the past, an artwork continues to exist in the present, long surviv- ing its times. The earliest known paintings and

In [9]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentences_count_spacy,num_chunks
count,3302.00,3302.00,3302.00,3302.00,3302.00,3302.00,3302.00
mean,475.77,3329.10,541.48,28.81,832.28,27.78,3.53
std,321.39,2035.33,320.31,33.24,508.83,27.95,3.11
min,0.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,206.00,2157.50,355.00,11.00,539.38,11.00,2.00
50%,416.00,2966.00,485.00,24.00,741.50,23.50,3.00
75%,724.75,4360.00,705.75,39.00,1090.00,38.00,5.00
max,1183.00,16445.00,2640.00,409.00,4111.25,356.00,40.00


**Разделение каждого чанка на отделиный элемент.**

In [10]:
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item['sentence_chunks']:
        chunk_dict = {}
        chunk_dict['source'] = item['pdf_file']
        chunk_dict['page_number'] = item['page_number']
        
        joined_sentence_chunk = ''.join(sentence_chunk).replace('  ', ' ').strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) 
        chunk_dict['sentence_chunk'] = joined_sentence_chunk

        chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
        chunk_dict['chunk_word_count'] = len([word for word in joined_sentence_chunk.split(' ')])
        chunk_dict['chunk_token_count'] = len(joined_sentence_chunk) / 4 
        
        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/3302 [00:00<?, ?it/s]

11656

**В итоге из 4 PDF-файлов нам удалось извлечь 11 656 чанков.**

**Теперь посмотрим на их качество.**

In [11]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,11656.00,11656.00,11656.00,11656.00
mean,547.55,239.91,38.79,59.98
std,341.48,398.35,63.53,99.59
min,0.00,2.00,1.00,0.50
25%,252.00,78.00,13.00,19.50
50%,504.50,154.00,25.00,38.50
75%,846.25,297.00,49.00,74.25
max,1183.00,10396.00,1688.00,2599.00


**Можно отметить, что в столбце chunk_token_count присутствуют значения, которые не представляют практической значимости: минимальное значение составляет 0.50, а максимальное достигает 2599.**

**Давайте внимательно рассмотрим эти чанки, чтобы понять их содержание.**

In [12]:
min_token_length = 30
for row in df[df['chunk_token_count'] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]['chunk_token_count']} | Text: {row[1]['sentence_chunk']}')

Chunk token count: 28.5 | Text: This chapter examines in turn Romanesque France and Spain, the Holy Roman Empire, Italy, and Normandy and England.
Chunk token count: 27.0 | Text: Pilliod, Elizabeth. El Greco. Women and Art in Early Modern Europe: Patrons, Col- lectors, and Connoisseurs.
Chunk token count: 19.5 | Text: Egyptian blue or “blue frit” is often described as the ﬁrst synthetic pigment.
Chunk token count: 18.0 | Text: Roman copy from the palaestra, Pompeii, Italy, of a bronze statue of ca.
Chunk token count: 29.75 | Text: The crossbow- men s guild of Louvain (near Brussels) commissioned it as the center of an altarpiece for a church there.


In [13]:
max_token_length = 500
for row in df[df['chunk_token_count'] >= max_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]['chunk_token_count']} | Text: {row[1]['sentence_chunk']}')

Chunk token count: 1514.75 | Text: Ravenna Mausoleum of Galla Placidia, 248 250, 248 250 San Vitale church, 254, 255 257, 255 256 Rayograph, 992 Rayonnant or Court style, 413 418, 413 417 use of term, 415 Readymades, 972, 986 Realism development of, 859 860 painting, American, 885 890, 886 890 painting, English, 881 883, 882, 884 887, 884, 886 painting, French, 860 866, 862 866, 868 870, 868 870 rejection of academic values, 866 scientific realism painting, 887 888, 888 sculpture, French, 866 867, 867 Recumbent Figure (Moore), 1002, 1002 Red Blue Green (Kelly), 1054, 1055 Red Studio (Matisse), 949, 949 Red-figure pottery style, 121 123, 122 Red-figured calyx krater (Niobid Painter), 146, 146 Redon, Odilon, Eye Like a Strange Balloon Mounts Toward Infinity, 918 919, 919 Reed Painter, White-ground lekythos, 146 147, 146 Reflections on the Imitation of Greek Art in Painting and Sculpture (Winckelmann), 787 Reformation, 591 592, 603, 625 art in central Europe, 634 646, 636 646 in Basel, 64

**Избавимся от этих чанков, так как они не содержат полезной информации и могут лишь снизить общее качество данных.**

In [14]:
pages_and_chunks_over = df[(df["chunk_token_count"] > min_token_length) & (df["chunk_token_count"] < max_token_length)].to_dict(orient="records")

### Создание векторов

**После того как мы подготовили наши чанки, займемся их преобразованием в векторное представление. В этом нам поможет библиотека sentence-transformers, которая предоставляет удобные инструменты для создания эмбеддингов текста с использованием предобученных моделей.**

In [15]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2', 
                                      device='cuda')

In [16]:
for item in tqdm(pages_and_chunks_over):
    item['embedding'] = embedding_model.encode(item['sentence_chunk'])

  0%|          | 0/6949 [00:00<?, ?it/s]

In [17]:
embedding_df = pd.DataFrame(pages_and_chunks_over)
embedding_df.sample(5)

,source,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
5913,Transformative Art Movements and the Paintings...,215,Self-Portrait With a Portrait of Her Sister Ro...,162,24,40.50,"[0.02697511, 0.08461836, -0.023276823, 0.01099..."
2000,Gardners Art Through the Ages The Western Pers...,704,"Francis favored art that was at once elegant, ...",135,24,33.75,"[0.037299544, 0.08872556, -0.041632187, 0.0723..."
6588,Vasari Giorgio_The_Lives_of_the_Artists_Oxford...,303,Preface to Part Three Those excellent masters ...,580,101,145.00,"[0.042665754, 0.038328882, -0.05292479, 0.0379..."
1694,Gardners Art Through the Ages The Western Pers...,599,598 Chapter 21 The Renaissance in Quattrocento...,793,138,198.25,"[0.03272, 0.03345174, 0.00044551038, 0.0578710..."
6160,Transformative Art Movements and the Paintings...,384,383 THE FIGURATIVE TRADITION MASTERWORK Beneﬁt...,277,44,69.25,"[0.071031764, 0.0033289467, -0.042039618, 0.01..."


In [18]:
embedding_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6949 entries, 0 to 6948
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   source             6949 non-null   object 
 1   page_number        6949 non-null   int64  
 2   sentence_chunk     6949 non-null   object 
 3   chunk_char_count   6949 non-null   int64  
 4   chunk_word_count   6949 non-null   int64  
 5   chunk_token_count  6949 non-null   float64
 6   embedding          6949 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 380.2+ KB


**Замечательно! Теперь у нас есть векторы чанков!**

**Для дальнейшей работы нам потребуется преобразовать наши векторные представления в тензоры.**

In [19]:
import torch
import numpy as np
from sentence_transformers import util

device = 'cuda' if torch.cuda.is_available() else 'cpu'

pages_and_chunks = embedding_df.to_dict(orient='records')

embeddings = torch.tensor(np.array(embedding_df['embedding'].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([6949, 768])

### Семантический поиск

**Теперь давайте проведем семантический поиск.**


**У нас есть два наиболее популярных метода для решения этой задачи:**
1. Использовать util из sentence transformers
2. FAISS

**Мы рассмотрим оба этих метода.**

### Работа с util

**Мы будем вычислять скалярное произведение между векторным представлением запроса и векторным представлением чанка. Поскольку оба вектора имеют одинаковую размерность, скалярное произведение позволит нам эффективно оценить степень их схожести!**

In [20]:
query = 'What was Claude Monet famous for?'
print(f'Query: {query}')

query_embedding = embedding_model.encode(query, convert_to_tensor=True).to(device)

dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]

top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: What was Claude Monet famous for?


torch.return_types.topk(
values=tensor([0.6220, 0.6093, 0.6052, 0.5918, 0.5864], device='cuda:0'),
indices=tensor([2429, 3179, 2444, 6175, 2436], device='cuda:0'))

**Супер! Мы получили отличные результаты. Теперь давайте проверим, насколько текст извлеченных чанков соответствует нашему запросу**

In [21]:
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [22]:
print(f"Query: '{query}'\n")
print("Results:")
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Source: {pages_and_chunks[idx]['source']}")
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

Query: 'What was Claude Monet famous for?'

Results:
Score: 0.6220
Text:
Claude Monet in His Studio Boat is also noteworthy as a document of Monet’s
preference for painting outdoors (en plein air)—a radical practice at the
time—in order to record his “impression” of the Seine by placing colors directly
on a white canvas without any preliminary sketch—also a sharp break from
traditional studio techniques.
Source: Gardners Art Through the Ages The Western Perspective, Volume I,.pdf
Page number: 850


Score: 0.6093
Text:
These include: Monet s Gare St. Lazare; Rossetti s Proserpine; Nadar s portrait
Édouard Manet; and Le Gray s Brig on the Water.
Source: History_of_Art.pdf
Page number: 16


Score: 0.6052
Text:
Monet’s intensive study of the phenomena of light and color is especially
evident in several series of paintings he made of the same subject late in his
career.
Source: Gardners Art Through the Ages The Western Perspective, Volume I,.pdf
Page number: 855


Score: 0.5918
Text:
Piet M

**Получилось отлично! Теперь давайте повторим это с FAISS.**

### Работа с FAISS

In [23]:
import faiss

embeddings = np.array(embedding_df['embedding'].tolist()).astype('float32')
dimension = embeddings.shape[1]  
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

In [24]:
query = 'What was Michelangelo famous for?'
query_embedding = embedding_model.encode(query, convert_to_tensor=False).astype('float32')
query_embedding = np.expand_dims(query_embedding, axis=0)

In [25]:
k = 5 
distances, indices = index.search(query_embedding, k)

print(f"Query: '{query}'\n")
print("Results:")
for score, idx in zip(distances[0], indices[0]):
    print(f"Score: {score:.4f}")
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Source: {pages_and_chunks[idx]['source']}")
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

Query: 'What was Michelangelo famous for?'

Results:
Score: 0.7094
Text:
472 MICHELANGELO \vas recognized during his lifetime and not, as happens to so
many, only after his death, for as we have seen, Julius II, Leo X, Clement VII,
Paul HI, Julius III, Paul IV, and Pius IV, all supreme pontiffs, -wished to have
him nearby at all times, and, as is well known, Suleiman, Emperor of the Turks,
Francis Valois, King of France, Emperor Charles V, the Signoria of Venice, and
finally Duke Cosimo de' Medici, as was mentioned, all provided him with generous
salaries for no other reason than to avail themselves of his great talent; this
happens only to men of great -worth, as he -was, for it was recognized and
understood that all three of these arts had reached a true state of perfection
in his works, and that God had not granted such genius either to the artists of
antiquity or to those of the modern period as He had to Michelangelo, in all the
many years the sun had been revolving.*
Source: Vasa

**Каждый из методов продемонстрировал отличные результаты! Благодаря этому мы успешно находим нужные фрагменты текста из нашего набора данных, соответствующие запросу пользователя.**

**Для удобства дальнейшей работы объединим все этапы в функции.**

In [26]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5):
 
    query_embedding = model.encode(query, convert_to_tensor=True).to(device)

    dot_scores = util.dot_score(query_embedding, embeddings)[0]

    scores, indices = torch.topk(input=dot_scores, k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    print(f"Query: '{query}'\n")
    print("Results:")
    for score, idx in zip(scores[0], indices[0]):
        print(f"Score: {score:.4f}")
        print("Text:")
        print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
        print(f"Source: {pages_and_chunks[idx]['source']}")
        print(f"Page number: {pages_and_chunks[idx]['page_number']}")
        print("\n")

### Добавление LLM

**В этой работе мы будем использовать модель LLaMA 3.2 с 3 миллиардами параметров.**

In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, TextStreamer

tokenizer = AutoTokenizer.from_pretrained('NousResearch/Hermes-3-Llama-3.2-3B', trust_remote_code=True)
llm_model = LlamaForCausalLM.from_pretrained(
    "NousResearch/Hermes-3-Llama-3.2-3B",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=False
)

streamer = TextStreamer(tokenizer)

llm_model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

**Давайте посмотрим, как модель генерирует ответ на запрос без использования retriever-а.**

In [28]:
input_text = "What was Claude Monet famous for?"
print(f"Input text:\n{input_text}")

dialogue_template = [
    {"role": "user",
     "content": input_text}
]

prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False, 
                                       add_generation_prompt=True)

input_ids = tokenizer(prompt, return_tensors="pt").to(device)
print(f"Model input (tokenized):\n{input_ids}\n")

outputs = llm_model.generate(**input_ids, streamer=streamer,
                             max_new_tokens=256)
print(f"Model output (tokens):\n{outputs[0]}\n")

Input text:
What was Claude Monet famous for?
Model input (tokenized):
{'input_ids': tensor([[128000, 128040,    882,    198,   3923,    574,  75430,   3206,    295,
          11495,    369,     30, 128039,    198, 128040,  78191,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|im_start|>user
What was Claude Monet famous for?<|im_end|>
<|im_start|>assistant
Claude Monet was a renowned French painter, considered one of the leading painters in the Impressionist movement. He is famous for his series of paintings depicting the effects of light and color on a scene, such as his famous series of Haystacks and Water Lilies. Monet's work is known for its focus on capturing the changing moods of a scene, often painting the same subject multiple times to show the effects of varying weather conditions and times of day.<|im_end|>
His works are characterized by their use of vibrant, pure colors and subtle variations of light. Monet's contributions to the development of Impressionism and his mastery of light and color have had a profound influence on art.<|im_end|>
His most famous works include "Impression, Sunrise" (Impression, soleil levant) (1872-1873), "Water Lilies" (Nymphéas) (1914-1926), "Haystacks" (La haye) (1890-1891), and "The Cathedral at Rou

In [29]:
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<|begin_of_text|>', '').replace('<|im_end|>', '')}")

Input text: What was Claude Monet famous for?

Output text:
Claude Monet was a renowned French painter, considered one of the leading painters in the Impressionist movement. He is famous for his series of paintings depicting the effects of light and color on a scene, such as his famous series of Haystacks and Water Lilies. Monet's work is known for its focus on capturing the changing moods of a scene, often painting the same subject multiple times to show the effects of varying weather conditions and times of day.
His works are characterized by their use of vibrant, pure colors and subtle variations of light. Monet's contributions to the development of Impressionism and his mastery of light and color have had a profound influence on art.
His most famous works include "Impression, Sunrise" (Impression, soleil levant) (1872-1873), "Water Lilies" (Nymphéas) (1914-1926), "Haystacks" (La haye) (1890-1891), and "The Cathedral at Rouen" (La cathédrale de Rouen) (1893-1895).
Monet's work is he

**Важным этапом в работе с языковыми моделями (LLM) является написание качественного промпта. Промпт (или подсказка) играет ключевую роль в том, как модель интерпретирует запрос и формирует ответ. Давайте займемся этим!**

In [55]:
def prompt_base(query: str, context_items: list[dict]) -> str:
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    base_prompt =  """
Привет, ты помогаешь пользователю разбираться в запросах по искусству. Отвечай на его вопросы, опираясь на контекст, отвечай на русском. Не пиши код, а словами объясняй как оно работает.
Постарайтесь, чтобы ваши ответы были как можно более пояснительными.
\nДля ответа на запрос пользователя используйте следующие контекстные элементы:
{context}
Запрос пользователя:
{query}
Ans:

"""

    base_prompt = base_prompt.format(context=context, query=query)

    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [56]:
query = "What was Claude Monet famous for?"
print(f"Query: {query}")

embeddings = torch.tensor(np.array(embedding_df['embedding'].tolist()), dtype=torch.float32).to(device)

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
    
context_items = [pages_and_chunks[i] for i in indices]

prompt = prompt_base(query=query,
                          context_items=context_items)
print(prompt)

Query: What was Claude Monet famous for?
<|im_start|>user

Привет, ты помогаешь пользователю разбираться в запросах по искусству. Отвечай на его вопросы, опираясь на контекст, отвечай на русском. Не пиши код, а словами объясняй как оно работает.
Постарайтесь, чтобы ваши ответы были как можно более пояснительными.

Для ответа на запрос пользователя используйте следующие контекстные элементы:
- Claude Monet in His Studio Boat is also noteworthy as a document of Monet’s preference for painting outdoors (en plein air)—a radical practice at the time—in order to record his “impression” of the Seine by placing colors directly on a white canvas without any preliminary sketch—also a sharp break from traditional studio techniques.
- These include: Monet s Gare St. Lazare; Rossetti s Proserpine; Nadar s portrait Édouard Manet; and Le Gray s Brig on the Water.
- Monet’s intensive study of the phenomena of light and color is especially evident in several series of paintings he made of the same subj

**Теперь давайте посмотрим, что сгенерирует модель с учетом промпта, который мы только что подготовили.**

In [57]:
input_ids = tokenizer(prompt, return_tensors="pt").to(device)

outputs = llm_model.generate(**input_ids,
                             temperature=0.7, 
                             do_sample=True,
                             streamer=streamer,
                             max_new_tokens=256) 

output_text = tokenizer.decode(outputs[0])

print(f"Query: {query}\n")
print(f"RAG answer:\n\n{output_text.replace(prompt, '')}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|im_start|>user

Привет, ты помогаешь пользователю разбираться в запросах по искусству. Отвечай на его вопросы, опираясь на контекст, отвечай на русском. Не пиши код, а словами объясняй как оно работает.
Постарайтесь, чтобы ваши ответы были как можно более пояснительными.

Для ответа на запрос пользователя используйте следующие контекстные элементы:
- Claude Monet in His Studio Boat is also noteworthy as a document of Monet’s preference for painting outdoors (en plein air)—a radical practice at the time—in order to record his “impression” of the Seine by placing colors directly on a white canvas without any preliminary sketch—also a sharp break from traditional studio techniques.
- These include: Monet s Gare St. Lazare; Rossetti s Proserpine; Nadar s portrait Édouard Manet; and Le Gray s Brig on the Water.
- Monet’s intensive study of the phenomena of light and color is especially evident in several series of paintings he made of the same subject late in his career.


### Результат

**Вот мы и завершили создание RAG! Осталось только взглянуть на конечный результат и оценить, насколько хорошо система справляется с задачей.**

In [70]:
def ask(query, 
        temperature=0.7,
        max_new_tokens=256,
        format_answer_text=True, 
        return_answer_only=True):

    scores, indices = retrieve_relevant_resources(query=query, embeddings=embeddings)
    

    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() 
        
    prompt = prompt_base(query=query, context_items=context_items)
    
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens,
                                eos_token_id=tokenizer.eos_token_id,  
                                pad_token_id=tokenizer.pad_token_id,  
                                no_repeat_ngram_size=2)
    
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        output_text = output_text.replace(prompt, "").replace("<|im_start|>", "").replace("<|im_end|>", "").replace("Sure, here is the answer to the user query:\n\n", "").replace("<|begin_of_text|>", "")

    if return_answer_only:
        return output_text
    
    return output_text

In [71]:
query = 'What was Claude Monet famous for?'
print(f"Query: {query}")

answer = ask(query=query, 
            temperature=0.7,
            max_new_tokens=256,
            return_answer_only=True)

print(f"Answer:\n")
print_wrapped(answer)

Query: What was Claude Monet famous for?
Answer:

Клод Моне был известен как один из основоположников импрессионистского движения
в искусстве. Он прославился своим оригинальным подходом к живописи, особенно
своим использованием "импрессии" (импр. от фр. "imprimer", то есть давать
впечатление) - то же, что и название движения. Моны часто работал на свежем
воздухе (по-французски "en pleine nature"), создавая живые и яркие картины,
которые отражают его чувство цвета и светла.  Его наиболее известные работы,
такие как "Гарь Ст. Лазар", стали символами импресиономизма. Клод также
известный своим интенсивным изучением явлений света и цвета, которое проявляется
в его сериях полотен одного и того же объекта поздней его карьеры.  Таким
образом, Монт известны не только своей оригинойльной техникой, но и своим
способностью передавать на полотне ощущение времени и места.


**Вау! Действительно, модель успешно извлекла и объединила ключевые детали из данных, несмотря на то, что LLaMA 3.2 с 3 миллиардами параметров не является самой крупной моделью. Это отличный результат, который демонстрирует, что даже относительно компактные модели могут эффективно справляться с задачами, если правильно настроены и используются в сочетании с такими подходами, как RAG.**